#Laboratorio 11 - QR decomposition of a matrix
##Métodos Numéricos
 * Julián David Puentes Gil
 * Gustavo Alberto Puentes Romero
 

In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [2]:
def householder(vec):
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)

    u= vec.copy()
    u[0] = (-vec[1:]**2).sum()/(vec[0] + np.linalg.norm(vec, ord = 2))
    u /=np.linalg.norm(u, ord = 2)
    
    H = np.eye(vec.shape[0])
    H-=2*u.reshape(-1,1)@u.reshape(1,-1)
    
    return np.dot(H, vec), H

Test your function using tests below:

In [3]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [4]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [5]:
def qr_decomp(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    Q = np.eye(m)

    
    for i in range(n):
        _, H = householder(a1[i:,i])
        h = np.eye(m)
        h[i:,i:] = H
        
        a1 = h @ a1
        Q = Q @ h
        
    return Q, a1


In [6]:
np.set_printoptions(suppress=True)

In [7]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# Test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e1)

# Test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [9]:
print("q: \n",q,"\n \n r: \n",r)

q: 
 [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 
 
 r: 
 [[ 1.40152769  1.2514379   0.89523615]
 [-0.          0.84158252  0.68447942]
 [ 0.         -0.          0.5496046 ]
 [-0.          0.         -0.        ]
 [-0.         -0.          0.        ]]


In [10]:
print("qq: \n",qq,"\n \n r: \n",rr)

qq: 
 [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 
 
 r: 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


*Enter your explanation here* (10% of the total grade, peer-graded)

**Matricies contain the same values in most of the times, excepts for the two lasts columns in $q$ and $qq$ that have differents values because the last two rows in $r$ has only zeros. In some cases we have diferents signs, thats because the matrix rotation.**

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [11]:
# Compute the QR decomposition of an m-by-n matrix A using
def qr_decomp_v2(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    R = a1.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        normx = np.linalg.norm(R[j:, j])
        if R[j, j] > 0:
            u1 = R[j, j] + normx
            tau = u1 / normx
        else:
            u1 = R[j, j] - normx
            tau = -u1 / normx 
        w = R[j:, j] / u1
        w[0] = 1
        w = np.array([w])

        R[j:, :] = R[j:, :] - (tau*w.T)@(w@R[j:, :])
        Q[:, j:] = Q[:, j:] - ((Q[:, j:]@w.T)@np.transpose(tau*w.T))
    
    return Q, R



In [12]:
#test Q@R = qq@rr
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
Q, R = qr_decomp_v2(a)
qq, rr = qr(a)

assert_allclose(np.dot(Q, R), np.dot(qq, rr))

In [13]:
def QTX(QR, tau, X):
    m, n = QR.shape
    QTX = X
    for j in range(0, n):
        w = [0, QR[j+1:,j]]
        w = np.array([w])
        QTX[j:,:] = QTX[j:,:]-[tau(j)@w.T]@[w@QTX[j:,:]]
    return QTX